# Scrapy
- 비동기 방식으로 데이터를 수집> 데이터 수집이 빠름

In [3]:
import scrapy, requests
from scrapy.http import TextResponse

In [ ]:
# Gmarket 베스트 200 상품 데이터 수집

In [ ]:
# 1. 스크레피 프로젝트 생성

In [4]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\Desktop\python\크롤링\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [5]:
!tree gmarket /f

폴더 PATH의 목록입니다.
볼륨 일련 번호가 000000EA 7C6C:FC35입니다.
C:\USERS\USER\DESKTOP\PYTHON\크롤링\GMARKET
│  scrapy.cfg
│  
└─gmarket
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


In [ ]:
# items.py: 수집할 데이터의 구조 정의
# middlewares.py:데이터를 수집할때 headers 정보와 같은 내용을 설정
# piplines.py: 데이터를 수집한 후 코드 실행 정의
# settings.py: 크롤링 설정: 크롤링 시간 텀, robots.txt 규칙
# spiders:크롤링 절차 정의

In [ ]:
# 2. xpath 찾기: 링크, 상세 페이지 데이터 

In [6]:
url ='http://corners.gmarket.co.kr/Bestsellers'

In [ ]:
# 링크데이터 수집

In [8]:
request = requests.get(url)
response = TextResponse(request.url,body = request.text,encoding='utf-8')

In [13]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
links[:2]

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953125844025224',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953125844025224']

In [10]:
# 상세 페이지 데이터 수집

In [16]:
link = links[0]
request = requests.get(link)
response = TextResponse(request.url,body = request.text,encoding='utf-8')

In [18]:
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()
title,price

('브라운 면도기 세정액 CCR5+1 ', '25,030')

In [ ]:
# 3. items.py:코드작성:model

In [19]:
%%writefile gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item):
    title =scrapy.Field()
    price =scrapy.Field()
    link =scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [ ]:
# 4. spider.py:코드작성: 크롤링 절차 정의

In [20]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Writing gmarket/gmarket/spiders/spider.py


In [ ]:
# 5. 스크레피 프로젝트 실행
# gmarket 디렉토리
# scrapy crawl GMB -o items.csv

In [21]:
%pwd

'C:\\Users\\User\\Desktop\\python\\크롤링'

In [23]:
pd.read_csv('gmarket/items.csv').tail(2)

,link,price,title
18,http://item.gmarket.co.kr/Item?goodscode=25204...,"28,800",(롯데백화점)남여공용 디스럽터 테이피 테잎 _ FS1HTB1091X
19,http://item.gmarket.co.kr/Item?goodscode=21674...,"72,000",닌자 프로페셔널 초퍼 믹서기 블렌더 NJ1001KR
